## Aim: To implement BFS,DFS and Best First Search


## Theory
To search for a solution a agent can use different searches
1. Informed search: have a idea of goal
   - best first search
3. Uninformed search: no idea of goal
   - breadth first search
   - depth first search

## Code

In [2]:
graph = {'A': set(['B', 'C']),
         'B': set(['A', 'D', 'E']),
         'C': set(['A', 'F']),
         'D': set(['B']),
         'E': set(['B', 'F']),
         'F': set(['C', 'E'])}

In [3]:
def dfs(graph, start):
    visited, stack = set(), [start] 
    while stack:
        vertex = stack.pop()
        if vertex not in visited:
            visited.add(vertex)
            stack.extend(graph[vertex] - visited)
    return visited

dfs(graph, 'A')


{'A', 'B', 'C', 'D', 'E', 'F'}

In [50]:


def dfs(graph, start):
    visited, stack = [], [start] 
    while stack:
        vertex = stack.pop()
        if vertex not in visited:
            visited.append(vertex)
            stack.extend(graph[vertex] - set(visited))
    return visited

dfs(graph, 'A') 

['A', 'B', 'E', 'F', 'C', 'D']

The second implementation provides the same functionality as the first, however, this time we are using the more succinct recursive form. Due to a common Python gotcha with default parameter values being created only once, we are required to create a new visited set on each user invocation. Another Python language detail is that function variables are passed by reference, resulting in the visited mutable set not having to reassigned upon each recursive call.



In [4]:
def dfs(graph, start, visited=None):
    if visited is None:
        visited = set().
    visited.add(start)
    for next in graph[start] - visited:
        dfs(graph, next, visited)
    return visited

dfs(graph, 'C')


{'A', 'B', 'C', 'D', 'E', 'F'}

In [51]:


def dfs(graph, start, visited=[]): 
    visited.append(start)
    for next in graph[start] - set(visited): 
        dfs(graph, next, visited)
    return visited

dfs(graph, 'C') 


['C', 'F', 'E', 'B', 'A', 'D', 'A']

## Paths

We are able to tweak both of the previous implementations to return all possible paths between a start and goal vertex. The implementation below uses the stack data-structure again to iteratively solve the problem, yielding each possible path when we locate the goal. Using a generator allows the user to only compute the desired amount of alternative paths.



In [43]:
def dfs_paths(graph, start, goal):
    stack = [(start, [start])]
    while stack:
        (vertex, path) = stack.pop()
        for next in graph[vertex] - set(path):
            if next == goal:
                yield path + [next]
            else:
                stack.append((next, path + [next]))

list(dfs_paths(graph, 'A', 'F')) 


[['A', 'B', 'E', 'F'], ['A', 'C', 'F']]

The implementation below uses the recursive approach calling the ‘yield from’ PEP380 addition to return the invoked located paths. Unfortunately the version of Pygments installed on the server at this time does not include the updated keyword combination.

In [42]:
def dfs_paths(graph, start, goal, path=None):
    if path is None:
        path = [start]
    if start == goal:
        yield path
    for next in graph[start] - set(path):
        yield from dfs_paths(graph, next, goal, path + [next])

list(dfs_paths(graph, 'C', 'F')) 


[['C', 'F'], ['C', 'A', 'B', 'E', 'F']]

# Breath-First Search

An alternative algorithm called Breath-First search provides us with the ability to return the same results as DFS but with the added guarantee to return the shortest-path first. This algorithm is a little more tricky to implement in a recursive manner instead using the queue data-structure, as such I will only being documenting the iterative approach. The actions performed per each explored vertex are the same as the depth-first implementation, however, replacing the stack with a queue will instead explore the breadth of a vertex depth before moving on. This behavior guarantees that the first path located is one of the shortest-paths present, based on number of edges being the cost factor.

Connected Component
Similar to the iterative DFS implementation the only alteration required is to remove the next item from the beginning of the list structure instead of the stacks last.



In [7]:
def bfs(graph, start):
    visited, queue = set(), [start]
    while queue:
        vertex = queue.pop(0)
        if vertex not in visited:
            visited.add(vertex)
            queue.extend(graph[vertex] - visited)
    return visited

bfs(graph, 'A')

{'A', 'B', 'C', 'D', 'E', 'F'}

In [46]:

def bfs(graph, start):
    visited, queue = [], [start] 
    while queue:
        vertex = queue.pop(0)
        if vertex not in visited:
            visited.append(vertex)
            queue.extend(graph[vertex] - set(visited)) 
    return visited

bfs(graph, 'A') 

['A', 'C', 'B', 'F', 'D', 'E']

# Paths

This implementation can again be altered slightly to instead return all possible paths between two vertices, the first of which being one of the shortest such path.



In [49]:
def bfs_paths(graph, start, goal):
    queue = [(start, [start])]
    while queue:
        (vertex, path) = queue.pop(0)
        for next in graph[vertex] - set(path):
            if next == goal:
                yield path + [next]
            else:
                queue.append((next, path + [next]))

list(bfs_paths(graph, 'A', 'F')) 


[['A', 'C', 'F'], ['A', 'B', 'E', 'F']]

Knowing that the shortest path will be returned first from the BFS path generator method we can create a useful method which simply returns the shortest path found or ‘None’ if no path exists. As we are using a generator this in theory should provide similar performance results as just breaking out and returning the first matching path in the BFS implementation.



In [9]:
def shortest_path(graph, start, goal):
    try:
        return next(bfs_paths(graph, start, goal))
    except StopIteration:
        return None

shortest_path(graph, 'A', 'F')

['A', 'C', 'F']

## Best First Search algorithm

In [1]:
import heapq

class Graph:
    def __init__(self):
        self.graph = {}
        self.heuristics = {}

    def add_edge(self, node, neighbor, cost):
        if node not in self.graph:
            self.graph[node] = []
        self.graph[node].append((neighbor, cost))

    def set_heuristic(self, node, heuristic):
        self.heuristics[node] = heuristic

def best_first_search(graph, start, goal):
    frontier = [(graph.heuristics[start], start, [start], 0)]  # (heuristic, node, path, cost)
    visited = set()

    while frontier:
        _, current, path, cost = heapq.heappop(frontier)

        if current == goal:
            return path, cost

        if current not in visited:
            visited.add(current)
            for neighbor, edge_cost in graph.graph.get(current, []):
                if neighbor not in visited:
                    new_path = path + [neighbor]
                    new_cost = cost + edge_cost
                    heapq.heappush(frontier, (graph.heuristics[neighbor], neighbor, new_path, new_cost))

    return [], float('inf')

# Small example
g = Graph()
g.add_edge('A', 'B', 3)
g.add_edge('A', 'C', 1)
g.add_edge('C', 'D', 4)
g.set_heuristic('A', 5)
g.set_heuristic('B', 2)
g.set_heuristic('C', 3)
g.set_heuristic('D', 0)

path, cost = best_first_search(g, 'A', 'D')
print(f"Path: {' -> '.join(path) if path else 'No path found'}")
print(f"Total cost: {cost}")

Path: A -> C -> D
Total cost: 5


## Result
1. DFS
{'A', 'B', 'C', 'D', 'E', 'F'}
2. BFS
{'A', 'B', 'C', 'D', 'E', 'F'}   
3. Best First Search
Path: A -> C -> D
Total cost: 5

## Learning Outcome
#### I learned to implement DFS, BFS, and best first search algorithms